In [1]:
# use hmark_env2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
from sklearn.linear_model import ElasticNetCV, ElasticNet
from sklearn.model_selection import cross_val_score, cross_val_predict, KFold
import numpy as np
#from xgboost import XGBRegressor
from scipy.stats import spearmanr, pearsonr
import requests
import glob

# turn on autoreload
%load_ext autoreload
%autoreload 2

source_path = os.path.abspath(os.path.join('..'))
if source_path not in sys.path:
    sys.path.append(os.path.join(source_path, 'source'))
# read source files–––
from histone_mark_dataset import histoneMarkDataset
from methylation_dataset import methylationDataset
import read_data

# Human/Mouse encode data

### Create metadata

In [44]:
experiments[0]['replicates'][0]['library']['biosample']

{'dbxrefs': ['GEO:SAMN40088291'],
 'aliases': ['roderic-guigo:td_BE1_H009_biosample'],
 'starting_amount_units': 'cells',
 'references': [],
 'originated_from': {'dbxrefs': [],
  'aliases': ['roderic-guigo:td_BE1_biosample'],
  'references': [],
  'documents': [],
  '@type': ['Biosample', 'Item'],
  'applied_modifications': [],
  'perturbed': False,
  'accession': 'ENCBS167WPF',
  'source': '/sources/thomas-graf/',
  'lab': '/labs/roderic-guigo/',
  'characterizations': [],
  'uuid': '68c37670-dcfe-4349-9448-6cfcb40884bf',
  'treatments': [],
  'schema_version': '26',
  'award': '/awards/ERC-2011-AdG-294653-RNA-MAPS/',
  'donor': '/human-donors/ENCDO305PTK/',
  'genetic_modifications': [],
  'alternate_accessions': [],
  '@id': '/biosamples/ENCBS167WPF/',
  'origin_batch': '/biosamples/ENCBS167WPF/',
  'summary': 'Homo sapiens BLaER1 cell line',
  'organism': '/organisms/human/',
  'date_created': '2022-01-27T21:47:19.529052+00:00',
  'sex': 'female',
  'submitted_by': '/users/15bae39d

In [131]:
species = "mouse"
if species == "human":
    species_query = "Homo+sapiens"
else:
    species_query = "Mus+musculus"
datatype = "histone"

# allow classification to be tissue or cell+line
if datatype == "histone":
    url = (
        'https://www.encodeproject.org/search/?type=Experiment'
        '&status=released'
        '&assay_title=Histone+ChIP-seq'   
        '&biosample_ontology.classification=tissue'
        '&biosample_ontology.classification=cell+line'
        f'&replicates.library.biosample.donor.organism.scientific_name={species_query}'
        '&frame=embedded'
        '&format=json'
        '&limit=all'
    )
else:
    url = (
        'https://www.encodeproject.org/search/?type=Experiment'
    '&status=released'
    '&assay_title=WGBS'   
    '&assay_title=DNAme+array'   
    '&biosample_ontology.classification=tissue'
    '&biosample_ontology.classification=cell+line'
    f'&replicates.library.biosample.donor.organism.scientific_name={species_query}'
    '&frame=embedded'
    '&format=json'
    '&limit=all'
    )

r = requests.get(url)
experiments = r.json()['@graph']
metadata_dict = {}
for e in experiments:
    # for each replicate
    for r in e['replicates']:
        sex, age, age_units, donor_accession = np.nan, np.nan, np.nan, np.nan
        # get sex
        sex = r['library']['biosample']['sex']
        # get age
        age = r['library']['biosample']['age']
        # get age_units
        try:
            age_units = r['library']['biosample']['age_units']
        except(KeyError):
            age_units = np.nan
        # get accession
        accession = r['library']['biosample']['accession']
        # get donor accession
        donor_accession = r['library']['biosample']['donor']['accession']
        # get tissue
        tissue = r['library']['biosample']['biosample_ontology']['term_name']
        
        metadata_dict[accession] = {
            'sex': sex, 'age': age, 'age_units': age_units, 'donor_accession': donor_accession, 'tissue': tissue
            }
        
metadata_df = pd.DataFrame(metadata_dict).T
metadata_df.set_index('donor_accession', inplace=True)
metadata_df = metadata_df.loc[~metadata_df.index.duplicated(keep='first')]

download_fn = os.path.join('/cellar/users/zkoch/histone_mark_proj/data/encode', f"{species}_{datatype}_download.txt")
download_df = pd.read_csv(download_fn, header=None, skiprows=0, sep='\t').rename(columns={0: 'url'})
download_df['File accession'] = download_df['url'].str.split('/').str[4]

manifest_fn = os.path.join('/cellar/users/zkoch/histone_mark_proj/data/encode/metadata', f"{species}_{datatype}_metadata_download.tsv")
manifest_df = pd.read_csv(manifest_fn, sep='\t')
manifest_df['donor'] = manifest_df['Donor(s)'].str.split('/').str[2]

downloaded_files = download_df['File accession'].unique()
manifest_of_downloaded_files = manifest_df.query("`File accession` in @downloaded_files")
manifest_of_downloaded_files.reset_index(drop=True, inplace=True)
manifest_of_downloaded_files = manifest_of_downloaded_files[['File accession', 'Biosample term name', 'Biosample type', 'Experiment target', 'donor']]
manifest_of_downloaded_files.rename(columns = {'Biosample term name': 'tissue', 'Biosample type':'sample_type', 'Experiment target': 'histone_mark'}, inplace=True)

manifest_of_downloaded_files['age'] = manifest_of_downloaded_files['donor'].map(metadata_df['age'])
manifest_of_downloaded_files['age_units'] = manifest_of_downloaded_files['donor'].map(metadata_df['age_units'])
manifest_of_downloaded_files['sex'] = manifest_of_downloaded_files['donor'].map(metadata_df['sex'])

# convert age to float, dropping nan first
manifest_of_downloaded_files.query("age != 'unknown'", inplace=True)
manifest_of_downloaded_files['age'] = manifest_of_downloaded_files['age'].replace('90 or above', '90').replace('2-4', np.nan)
manifest_of_downloaded_files['age'] = manifest_of_downloaded_files['age'].astype(float)
metadata_df.dropna(subset=['age'], inplace=True)

# for ages with units of weeks, divide by 52
manifest_of_downloaded_files['age_years'] = manifest_of_downloaded_files['age']
manifest_of_downloaded_files.loc[manifest_of_downloaded_files['age_units'] == 'week', 'age_years'] = manifest_of_downloaded_files.loc[manifest_of_downloaded_files['age_units'] == 'week', 'age_years'] / 52
# for ages with units of days, divide by 365
manifest_of_downloaded_files.loc[manifest_of_downloaded_files['age_units'] == 'day', 'age_years'] = manifest_of_downloaded_files.loc[manifest_of_downloaded_files['age_units'] == 'day', 'age_years'] / 365
# for ages with units of months, divide by 12
manifest_of_downloaded_files.loc[manifest_of_downloaded_files['age_units'] == 'month', 'age_years'] = manifest_of_downloaded_files.loc[manifest_of_downloaded_files['age_units'] == 'month', 'age_years'] / 12
# drop age_units
manifest_of_downloaded_files.drop(columns=['age_units', 'age'], inplace=True)
# create histone column
if datatype == "histone":
    manifest_of_downloaded_files['histone_mark'] = manifest_of_downloaded_files['histone_mark'].str.split('-').str[0]
else:
    # drop histone_mark column
    manifest_of_downloaded_files.drop(columns=['histone_mark'], inplace=True)

manifest_of_downloaded_files.to_parquet(os.path.join('/cellar/users/zkoch/histone_mark_proj/data/encode/metadata', f"{species}_{datatype}_metadata_fixed.parquet"))

/tmp/ipykernel_474496/1191992623.py:71: DtypeWarning: Columns (27,58) have mixed types. Specify dtype option on import or set low_memory=False.
  manifest_df = pd.read_csv(manifest_fn, sep='\t')


In [135]:
manifest_of_downloaded_files#['donor'].value_counts()

,File accession,tissue,sample_type,histone_mark,donor,sex,age_years
0,ENCFF823VWF,heart,tissue,H3K4me3,ENCDO956IXV,mixed,0.039726
2,ENCFF551GXY,erythroblast,primary cell,H3K9me3,ENCDO083AAA,NaN,NaN
3,ENCFF861SPX,G1E,cell line,H3K9me3,ENCDO089AAA,male,0.000000
5,ENCFF682TYN,embryonic fibroblast,primary cell,H3K27ac,ENCDO072AAA,NaN,NaN
6,ENCFF728FZO,erythroblast,primary cell,H3K27me3,ENCDO083AAA,NaN,NaN
...,...,...,...,...,...,...,...
650,ENCFF858DIX,liver,tissue,H3K9me3,ENCDO469XMJ,mixed,0.039726
651,ENCFF188ILR,embryonic facial prominence,tissue,H3K36me3,ENCDO469XMJ,mixed,0.039726
652,ENCFF174MOQ,lung,tissue,H3K9me3,ENCDO956IXV,mixed,0.039726
653,ENCFF736SSJ,limb,tissue,H3K27me3,ENCDO956IXV,mixed,0.039726


# Blueprint

In [89]:
blueprint_metadata_fn = "/cellar/users/zkoch/histone_mark_proj/data/blueprint/blueprint_file_metadata.tsv"
blueprint_metadata_df = pd.read_csv(blueprint_metadata_fn, sep="\t")
interesting_cols = [
    'SAMPLE_ID', # ERS954093
    'SAMPLE_NAME', # S00XDKU1
    'DONOR_ID', # 15548, WR27 # same as SAMPLE_BARCODE
    'DONOR_AGE', # 55 - 60
    'DONOR_SEX', # Male, Female
    'SAMPLE_SOURCE', # cell line
    'EXPERIMENT_TYPE',
    'DISEASE',
    'BIOMATERIAL_TYPE',
    'TISSUE_TYPE',
    'FILE', # 'blueprint/data/homo_sapiens/GRCh38/bone_marrow/15548/Multiple_Myeloma/Bisulfite-Seq/CNAG/S00XDKU1.CPG_methylation_calls.bs_call.GRCh38.20160531.bw'
    'FILE_TYPE'
    ]
blueprint_metadata_df = blueprint_metadata_df[interesting_cols]

exp_types = ['DNA Methylation', 'H3K27ac','H3K4me3', 'H3K4me1', 'H3K36me3', 'H3K9me3', 'H3K27me3' ]
blueprint_metadata_df.query("EXPERIMENT_TYPE in @exp_types", inplace=True)

# get rid of cell lines
blueprint_metadata_df.query("SAMPLE_SOURCE != 'Cell Line'", inplace=True)

# get samples with a non nan age
blueprint_metadata_df = blueprint_metadata_df.loc[blueprint_metadata_df['DONOR_AGE'].notna()]

# fill nan disease with 'Healthy'
blueprint_metadata_df['DISEASE'] = blueprint_metadata_df['DISEASE'].fillna('Healthy')

# BS_METH_CALL_CNAG is bw, but can convert to bed
file_types = ['CHIP_MACS2_BROAD_BED', 'CHIP_MACS2_BED', 'BS_METH_CALL_CNAG']
blueprint_metadata_df.query("FILE_TYPE in @file_types", inplace=True)

In [90]:
# find donors with DNA Methylation and at least 2 experiments
donors_w_dnam = blueprint_metadata_df.query("EXPERIMENT_TYPE == 'DNA Methylation'")['DONOR_ID'].unique()
donors_w_histone = blueprint_metadata_df.query("EXPERIMENT_TYPE in @exp_types and EXPERIMENT_TYPE != 'DNA Methylation'")['DONOR_ID'].unique()
print("Number of donors with DNA Methylation and histone marks: ", len(set(donors_w_histone).intersection(set(donors_w_dnam))))

Number of donors with DNA Methylation and histone marks:  100


# HistoneMarkDataset

In [98]:
# 12 min for all at 16 cores
human_encode_hmark = histoneMarkDataset(
    name = "human_encode_hmark",
    species = "homo_sapiens",
    narrowpeak_dir = "/cellar/users/zkoch/histone_mark_proj/data/encode/human_bed_files/histone",
    regulatory_features_path = "/cellar/users/zkoch/histone_mark_proj/data/ensembl_reference/Homo_sapiens.GRCh38.regulatory_features.v112.gff3.gz",
    gene_locations_path = "/cellar/users/zkoch/histone_mark_proj/data/ensembl_reference/Homo_sapiens.GRCh38.112.gtf.gz", 
    metadata_fn = os.path.join('/cellar/users/zkoch/histone_mark_proj/data/encode/metadata', f"human_histone_metadata_final.parquet"),
    n_cores = 16
)
human_encode_hmark.create_data_matrix()


Reading narrowPeak files: 100%|██████████| 2515/2515 [00:28<00:00, 88.04it/s] 


Reading regulatory features files


Creating data matrix: 100%|██████████| 2515/2515 [04:48<00:00,  8.71it/s]


In [99]:
# 3 min for all at 16 cores
mouse_encode_hmark = histoneMarkDataset(
    name = "mouse_encode_hmark",
    species = "mus_musculus",
    narrowpeak_dir = "/cellar/users/zkoch/histone_mark_proj/data/encode/mouse_bed_files2/histone",
    regulatory_features_path = "/cellar/users/zkoch/histone_mark_proj/data/ensembl_reference/mus_musculus.GRCm38.Regulatory_Build.regulatory_features.20180516.gff.gz",
    gene_locations_path = "/cellar/users/zkoch/histone_mark_proj/data/ensembl_reference/Mus_musculus.GRCm38.102.gtf.gz", 
    n_cores = 16
)
mouse_encode_hmark.create_data_matrix()

Reading narrowPeak files: 100%|██████████| 655/655 [00:00<00:00, 202810.36it/s]


Reading regulatory features files


Creating data matrix: 100%|██████████| 655/655 [01:09<00:00,  9.38it/s]


In [38]:
human_hist_metadata_df = pd.read_csv(os.path.join('/cellar/users/zkoch/histone_mark_proj/data/encode/metadata', "human_metadata.csv")).rename(columns = {'Unnamed: 0': 'File accession'})


# Wang 2017


In [2]:
# Method 1: Using pandas
with pd.HDFStore('/cellar/users/zkoch/histone_mark_proj/data/wang_2017/data/AllData_combatnonpara_logage.h5', 'r') as store:
    print("Available keys:", store.keys())

Available keys: ['/covariates', '/methy_mat']


In [3]:
cov_df = pd.read_hdf("/cellar/users/zkoch/histone_mark_proj/data/wang_2017/data/AllData_non_normalized_data_logage.h5", key = 'covariates')

In [4]:
df = cov_df['age_days'].value_counts().sort_index().reset_index()
df['unlogged_age_days'] = 2**df['age_days']
df['age_months'] = df['unlogged_age_days'] / 31
df

,age_days,count,unlogged_age_days,age_months
0,2.807355,10,7.00,0.225806
1,4.392317,9,21.00,0.677419
2,5.807355,10,56.00,1.806452
3,5.926948,12,60.84,1.962581
4,5.977280,38,63.00,2.032258
5,6.926948,2,121.68,3.925161
6,7.129283,51,140.00,4.516129
7,7.614710,15,196.00,6.322581
8,7.761551,4,217.00,7.000000
9,9.386380,20,669.24,21.588387


In [5]:
# read in from .h5
methyl_df = pd.read_hdf("/cellar/users/zkoch/histone_mark_proj/data/wang_2017/data/AllData_non_normalized_data_logage.h5", key = 'methy_mat')

# Hillje

In [41]:

import os
import subprocess
from pathlib import Path
import GEOparse

gse = GEOparse.get_GEO(geo="GSE179090",
                      destdir="/cellar/users/zkoch/histone_mark_proj/data/hillje_2022")

20-Dec-2024 11:16:00 DEBUG utils - Directory /cellar/users/zkoch/histone_mark_proj/data/hillje_2022 already exists. Skipping.
20-Dec-2024 11:16:00 INFO GEOparse - Downloading ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE179nnn/GSE179090/soft/GSE179090_family.soft.gz to /cellar/users/zkoch/histone_mark_proj/data/hillje_2022/GSE179090_family.soft.gz
100%|██████████| 7.69k/7.69k [00:00<00:00, 23.6kB/s]
20-Dec-2024 11:16:01 DEBUG downloader - Size validation passed
20-Dec-2024 11:16:01 DEBUG downloader - Moving /tmp/tmp_95cneib to /cellar/users/zkoch/histone_mark_proj/data/hillje_2022/GSE179090_family.soft.gz
20-Dec-2024 11:16:01 DEBUG downloader - Successfully downloaded ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE179nnn/GSE179090/soft/GSE179090_family.soft.gz
20-Dec-2024 11:16:01 INFO GEOparse - Parsing /cellar/users/zkoch/histone_mark_proj/data/hillje_2022/GSE179090_family.soft.gz: 
20-Dec-2024 11:16:01 DEBUG GEOparse - DATABASE: GeoMiame
20-Dec-2024 11:16:01 DEBUG GEOparse - SERIES: GSE179090

In [18]:
metadata_df = pd.read_csv("/cellar/users/zkoch/histone_mark_proj/data/hillje_2022/SraRunTable.csv")
metadata_df.rename(columns = {'Run': 'donor', 'AGE': 'age_months', 'chip_antibody': 'histone_mark', 'treatment':'diet'}, inplace=True)
metadata_df['age_years'] = metadata_df['age_months'].str.replace(' months', '').astype(float) / 52
metadata_df.set_index('donor', inplace=True)
# remove index name
metadata_df.index.name = None
metadata_df['donor'] = metadata_df.index
metadata_df = metadata_df[['donor','age_years', 'histone_mark', 'tissue', 'diet']]
metadata_df.to_csv("/cellar/users/zkoch/histone_mark_proj/data/hillje_2022/metadata.csv", index=True)

In [30]:
# 12 min for all at 16 cores
hillje = histoneMarkDataset(
    name = "hillje",
    species = "mouse",
    narrowpeak_dir = "/cellar/users/zkoch/histone_mark_proj/data/hillje_2022/broad_peaks",
    regulatory_features_path = "/cellar/users/zkoch/histone_mark_proj/data/ensembl_reference/mus_musculus.GRCm38.Regulatory_Build.regulatory_features.20180516.gff.gz",
    gene_locations_path = "/cellar/users/zkoch/histone_mark_proj/data/ensembl_reference/Mus_musculus.GRCm38.102.gtf.gz", 
    metadata_df = pd.read_csv("/cellar/users/zkoch/histone_mark_proj/data/hillje_2022/metadata.csv", index_col=0),
    n_cores = 16,
    split_on = '_',
    overwrite_cached_data_matrix = True,
    use_cached_data_matrix = False
)

Reading from parquet files


Reading narrowPeak files: 100%|██████████| 108/108 [00:00<00:00, 95045.08it/s]


Reading regulatory features files


Populating data matrix: 100%|██████████| 108/108 [00:01<00:00, 88.82it/s]


# Stubbs

In [21]:

stubbs_dir = "/cellar/users/zkoch/histone_mark_proj/data/stubbs_2017"
metadata_df = pd.read_csv(os.path.join(stubbs_dir, "metadata.csv"))

In [20]:
# Create metadata DataFrame from filenames
files = glob.glob(os.path.join(stubbs_dir, "*.bed.gz"))
metadata = []

for f in files:
    basename = os.path.basename(f)
    # Split filename to get components
    parts = basename.replace(".bed.gz", "").split("_")
    
    # Extract metadata
    sample_id = parts[0]
    age = int(parts[1].replace("wk", ""))  # Remove 'wk' and convert to int
    tissue = parts[2]
    
    metadata.append({
        'id': basename.replace(".bed.gz", ""),
        'age_weeks': age,
        'tissue': tissue
    })
# Create DataFrame and sort
metadata_df = pd.DataFrame(metadata)
metadata_df = metadata_df.sort_values(['age_weeks', 'tissue']).reset_index(drop=True)

# Save metadata
metadata_path = os.path.join(stubbs_dir, "metadata.csv")
metadata_df.to_csv(metadata_path, index=False)

print("Metadata preview:")
print(metadata_df.head())
print("\nSaved to:", metadata_path)

metadata_df['age_years'] = metadata_df['age_weeks'] / 52
metadata_df.rename(columns = {'id': 'donor'}, inplace=True)
metadata_df.drop(columns=['age_weeks'], inplace=True)
metadata_df.set_index('donor', inplace=True)
metadata_df['donor'] = metadata_df.index
metadata_df.to_csv(os.path.join(stubbs_dir, "metadata.csv"), index=False)


Metadata preview:
                 id  age_weeks  tissue
0  M02NB_1wk_Cortex          1  Cortex
1  M01NB_1wk_Cortex          1  Cortex
2  M03NB_1wk_Cortex          1  Cortex
3  M04NB_1wk_Cortex          1  Cortex
4   M01NB_1wk_Heart          1   Heart

Saved to: /cellar/users/zkoch/histone_mark_proj/data/stubbs_2017/metadata.csv


In [ ]:
# Initialize methylation dataset
stubbs_methyl = methylationDataset(
    name="stubbs_methyl",
    species="mouse",
    bedmethyl_dir=stubbs_dir,
    regulatory_features_path="/cellar/users/zkoch/histone_mark_proj/data/ensembl_reference/mus_musculus.GRCm38.Regulatory_Build.regulatory_features.20180516.gff.gz",
    gene_locations_path="/cellar/users/zkoch/histone_mark_proj/data/ensembl_reference/Mus_musculus.GRCm38.102.gtf.gz",
    metadata_df=metadata_df,
    n_cores=20,
    use_cached_data_matrix=False,
    overwrite_cached_data_matrix=True
)

In [33]:
df = pd.read_parquet('/cellar/users/zkoch/histone_mark_proj/data/stubbs_2017/M00018754_27wk_Lung_mean_signal.parquet')

# CEEHRC

In [ ]:
df = pd.read_csv("/cellar/users/zkoch/histone_mark_proj/data/CEEHRC/CEERHC_metadata.csv")
df = df[['id', 'cell_type', 'donor_age', 'donor_age_unit', 'tissue_type']]
df.rename(columns = {'id': 'donor', 'tissue_type': 'tissue', 'donor_age': 'age_years'}, inplace = True)
# drop rows where age_years is not 'year'
df = df[df['donor_age_unit'] == 'year']
# drop rows where age_years is == 90+
df = df[df['age_years'] != '90+']
# convert rows where age_years has '-' or ' - ' to be the average of the two values on either side of the dash
# for other rows just convert to float
df['age_years'] = df['age_years'].apply(lambda x: float(x) if not isinstance(x, str) or '-' not in x else float(sum([float(i.strip()) for i in x.replace(' ', '').split('-')])/2))
# combine cell_type and tissue to make a tissue column, using tissue where both are available, otherwise use cell_type
df['tissue'] = df.apply(lambda row: row['tissue'] if pd.notna(row['tissue']) else row['cell_type'], axis=1)
# drop cell_type column
df.drop(columns = ['cell_type'], inplace = True)
# drop donor_age_unit column
df.drop(columns = ['donor_age_unit'], inplace = True)
# drop nans
df.dropna(inplace = True)

# Get all parquet files in the data directories
import glob
import os

methyl_parquet_files = glob.glob('/cellar/users/zkoch/histone_mark_proj/data/CEEHRC/methyl_bed/*methylation_profile.parquet')
histone_parquet_files = glob.glob('/cellar/users/zkoch/histone_mark_proj/data/CEEHRC/bed_peaks/*.parquet')

# Create lists to store the parsed values
samples = []
donors = []
marks = []

# Parse each filename
for f in methyl_parquet_files:
    parts = os.path.basename(f).split('.')
    samples.append(parts[0])
    donors.append(parts[2]) 
    marks.append('methylation')
for f in histone_parquet_files:
    parts = os.path.basename(f).split('.')
    samples.append(parts[0])
    donors.append(parts[2])
    marks.append(parts[3])

# Create dataframe
files_df = pd.DataFrame({
    'sample': samples,
    'donor': donors,
    'mark': marks
})
# merge with metadata on donor
new_metadata_df = files_df.merge(df, on = 'donor', how = 'outer').dropna()
new_metadata_df.set_index('sample', inplace = True)
new_metadata_df.to_csv("/cellar/users/zkoch/histone_mark_proj/data/CEEHRC/metadata_final.csv", index = True)

In [86]:
ceehrc_hmark = histoneMarkDataset(
            name = "ceehrc_hmark",
            species = "human",
            narrowpeak_dir = "/cellar/users/zkoch/histone_mark_proj/data/CEEHRC/bed_peaks",
            regulatory_features_path = "/cellar/users/zkoch/histone_mark_proj/data/ensembl_reference/Homo_sapiens.GRCh38.regulatory_features.v112.gff3.gz",
            gene_locations_path = "/cellar/users/zkoch/histone_mark_proj/data/ensembl_reference/Homo_sapiens.GRCh38.112.gtf.gz",
            sample_name_part = 2,
            split_on = '.',
            metadata_df = metadata_df,
            n_cores = 8,
            use_cached_data_matrix = True,
            overwrite_cached_data_matrix = False,
            )

Reading cached data matrix from /cellar/users/zkoch/histone_mark_proj/data/CEEHRC/bed_peaks/data_matrix/ceehrc_hmark_data_matrix_w_meta.parquet


In [2]:
metadata_df = pd.read_csv("/cellar/users/zkoch/histone_mark_proj/data/CEEHRC/metadata_final.csv", index_col=0)


In [ ]:
ceehrc_methyl = methylationDataset(
    name = "ceehrc_methyl",
    species = "human",
    bedmethyl_dir = "/cellar/users/zkoch/histone_mark_proj/data/CEEHRC/methyl_bed",
    regulatory_features_path = "/cellar/users/zkoch/histone_mark_proj/data/ensembl_reference/Homo_sapiens.GRCh38.regulatory_features.v112.gff3.gz",
    gene_locations_path = "/cellar/users/zkoch/histone_mark_proj/data/ensembl_reference/Homo_sapiens.GRCh38.112.gtf.gz",
    metadata_df = metadata_df,
    n_cores = 12,
    use_cached_data_matrix = False,
    overwrite_cached_data_matrix = True,
    sample_name_part = 2,
    split_on = '.',
    start_sample_number = 0,
    end_sample_number = 2
    )

# Petkovich

In [146]:
metadata_df = pd.read_csv("/cellar/users/zkoch/histone_mark_proj/data/petkovich_2017/GSE80672_series_matrix.txt.gz", skiprows=34, sep='\t', header=None)
# transpose metadata_df
metadata_df = metadata_df.T
# set first row as header
metadata_df.columns = metadata_df.iloc[0]
# drop first row
metadata_df = metadata_df.iloc[1:]

metadata_df = metadata_df[['!Sample_geo_accession', '!Sample_source_name_ch1', '!Sample_characteristics_ch1' ]]
metadata_df.columns = ['donor', 'tissue', 'tissue2', 'strain2', 'age_months', 'sex', 'strain', 'diet']
metadata_df.drop(columns = ['tissue2', 'strain2'], inplace = True)
metadata_df['age_years'] = metadata_df['age_months'].str.split(':').str[-1].str.split('(').str[0].str.strip().astype(float) / 12
metadata_df.drop(columns = ['age_months', 'sex'], inplace = True)
metadata_df['strain'] = metadata_df['strain'].str.split(': ').str[-1].str.strip()
metadata_df['diet'] = metadata_df['diet'].str.split(':').str[-1].str.strip().map({'standard': 'standard', 'Calorie Restricted': 'CR'})
metadata_df.set_index('donor', inplace = True)
metadata_df['donor'] = metadata_df.index
metadata_df.index.name = None
metadata_df.to_csv("/cellar/users/zkoch/histone_mark_proj/data/petkovich_2017/metadata_final.csv", index = True)

In [ ]:
metadata_df = pd.read_csv("/cellar/users/zkoch/histone_mark_proj/data/petkovich_2017/metadata_final.csv", index_col=0)
petkovich_methyl = methylationDataset(
    name = "petkovich",
    species = "mouse",
    bedmethyl_dir = "/cellar/users/zkoch/histone_mark_proj/data/petkovich_2017/bed_methyl",
    regulatory_features_path = "/cellar/users/zkoch/histone_mark_proj/data/ensembl_reference/mus_musculus.GRCm38.Regulatory_Build.regulatory_features.20180516.gff.gz",
    gene_locations_path = "/cellar/users/zkoch/histone_mark_proj/data/ensembl_reference/Mus_musculus.GRCm38.102.gtf.gz",
    metadata_df = metadata_df,
    n_cores = 18,
    use_cached_data_matrix = False,
    overwrite_cached_data_matrix = True,
    sample_name_part = 0,
    split_on = '_',
    )

In [3]:
loader = read_data.DatasetLoader(
    dataset_name = "petkovich",
)
petkovich = loader.load_dataset()

Loading dataset: petkovich
Reading cached data matrix from /cellar/users/zkoch/histone_mark_proj/data/petkovich_2017/bed_methyl/data_matrix/petkovich_data_matrix_w_meta.parquet


# Meer

In [47]:
metadata_df = pd.read_csv(
                    '/cellar/users/zkoch/histone_mark_proj/data/meer_2018/GSE121141_series_matrix.txt',
                    skiprows=34,
                    sep='\t',
                    header=None
                    ).T
metadata_df.columns = metadata_df.iloc[0]
metadata_df = metadata_df.iloc[1:]
metadata_df = metadata_df[["ID_REF", "!Sample_characteristics_ch1"]]
metadata_df.columns = ['donor', 'strain2', 'sex', 'age_months', 'strain', 'tissue']
metadata_df.drop(columns = ['strain2', 'sex',  'strain',], inplace = True)
metadata_df['age_years'] = metadata_df['age_months'].str.split(':').str[-1].str.strip().astype(float) / 12
metadata_df.drop(columns = ['age_months'], inplace = True)
metadata_df['tissue'] = metadata_df['tissue'].str.split(':').str[-1].str.strip()
metadata_df.set_index('donor', inplace = True)
metadata_df['donor'] = metadata_df.index
metadata_df.index.name = None
metadata_df.to_csv("/cellar/users/zkoch/histone_mark_proj/data/meer_2018/metadata_final.csv", index = True)

In [ ]:
metadata_df = pd.read_csv("/cellar/users/zkoch/histone_mark_proj/data/meer_2018/metadata_final.csv", index_col=0)
# Initialize methylation dataset
meer = methylationDataset(
    name="meer",
    species="mouse",
    bedmethyl_dir="/cellar/users/zkoch/histone_mark_proj/data/meer_2018/bed_methyl",
    regulatory_features_path="/cellar/users/zkoch/histone_mark_proj/data/ensembl_reference/mus_musculus.GRCm38.Regulatory_Build.regulatory_features.20180516.gff.gz",
    gene_locations_path="/cellar/users/zkoch/histone_mark_proj/data/ensembl_reference/Mus_musculus.GRCm38.102.gtf.gz",
    metadata_df=metadata_df,
    n_cores=20,
    use_cached_data_matrix=False,
    overwrite_cached_data_matrix=True,
    split_on = '_',
    sample_name_part = 0,
)